In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
    

from AnimalShelter1 import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = '34475'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}, limit = 5000))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

    
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Stop all previously running Dash server threads
if hasattr(app, "_server_threads"):
    for t in list(app._server_threads.values()):
        try:
            t.kill()
        except Exception:
            pass
    app._server_threads = {}  # clear the thread registry


# Logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()


# Define the rescue types
rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

#App layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    html.Img(
        src=f'data:image/png;base64,{encoded_image}',
        style={'height':'150px', 'width':'auto', 'display':'block', 'margin':'0 auto'}
    ),
    
    
    html.Center(html.B(html.H1('CS-340 Dashboard - Brycen McEuen'))),
    html.Hr(),

    # Filtering options
    html.Div([
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[{'label': i, 'value': i} for i in rescue_types],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        ),
        html.Hr()
    ]),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records')
    ),

    html.Br(),
    html.Hr(),

    # Graph and Map side by side
    html.Div([
        html.Div(id='graph-id', style={"flex": "1", "height": "500px"}),
        html.Div(id='map-id', style={"flex": "1", "height": "500px"})
    ], style={'display': 'flex'})

])


#############################################
# Interaction Between Components / Controller
#############################################



# Callback for radio buttons    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-type-filter', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                & (df.sex_upon_outcome == 'Intact Female')
                & (df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156)]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & (df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156)]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & (df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300)]
     
    return dff.to_dict('records')
        
# Callback for pie graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData) if viewData else df

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
# Callback for sytles
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # handles None or empty list
        return []
    
    return [
        {
            'if': {'column_id': i},
            'backgroundColor': '#D2F3FF'
        } 
        for i in selected_columns
    ]


# Callback for map function
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    df = pd.DataFrame(viewData)
    
    if df.empty or not selected_rows:  # guard against empty/None
        return dl.Map(center=[37.77, -122.42], zoom=10, children=[
            dl.TileLayer()
        ], style={"width": "100%", "height": "500px"})
    
    row = selected_rows[0]
    
    # Make sure your DataFrame has enough columns
    if df.shape[1] <= 14:
        return dl.Map(center=[37.77, -122.42], zoom=10, children=[
            dl.TileLayer()
        ], style={"width": "100%", "height": "500px"})
    
    lat = df.iloc[row, 13]
    lon = df.iloc[row, 14]
    
    return dl.Map(center=[lat, lon], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[lat, lon], children=dl.Tooltip("Rescue Location"))
    ], style={"width": "100%", "height": "500px"})



app.run_server(debug=True)

Dash app running on http://127.0.0.1:29017/
